In [1]:
import pandas as pd
import numpy as np
import os
import glob
from glob import glob
import re
import luigi
from operator import itemgetter
from persiantools import characters
from persiantools.jdatetime import JalaliDate
import persiantools
import datetime
from re import search
import itertools
from statistics import mode
import datetime as dt
from joblib import Parallel, delayed
import chardet
import requests
from bs4 import BeautifulSoup
import json
import time
import io
import jdatetime
import TSETMCScraping
import ar_to_fa
import clean_cols
import str_to_date
import str_to_jdate
import str_to_time
import cleaning_path

In [ ]:
class TSETMC_Sraping(luigi.Task):
    read_path= luigi.Parameter(default=None)
    dirr_path= luigi.Parameter(default=None)
    def requiers(self):
        return luigi.LocalTarget(self.dirr_path)
    def output(self):
        return luigi.LocalTarget(self.dirr_path)
    def run(self):
        if os.path.isfile(self.dirr_path):
            if self.read_path ==None:
                pass
            else:
                read_link= pd.read_excel(self.read_path)
                df= TSETMCScraping.stock_information(read_link)
                df.to_csv(self.output().path, mode='a',index=False, header=None)
        else:
            read_link= pd.read_excel(self.read_path)
            df= TSETMCScraping.stock_information(read_link)
            df.to_csv(self.output().path, index=False,encoding='utf-8-sig')


In [ ]:
class TSETMC_Sraping_op(luigi.Task):
    read_path= luigi.Parameter(default=None)
    dirr_path= luigi.Parameter(default=None)
    def requiers(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.dirr_path)
    def run(self):
        Data_scraping= input('wich data do you want to scrap: (trade\information\shareholders) ')
        if Data_scraping =='information':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.stock_information(scrap_links)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')
        if Data_scraping == 'trade':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.stock_information(scrap_links)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')
        if Data_scraping == 'shareholders':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.stock_information(scrap_links)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')

In [ ]:
class cleaning (luigi.Task):
    dirr_path_2= luigi.Parameter(default=None)
    def requiers(self):
        return TSETMC_Sraping_op()
    def output(self):
        return luigi.LocalTarget(self.dirr_path_2)
    def run(self):
        if os.path.exists():
            pass
        else:
            df= pd.read_csv(self.input())
            df= cleaning_path.clean_cols(df)
            df= cleaning_path.ar_to_fa(df)
            df= cleaning_path.str_to_date(df)
            df= cleaning_path.date_to_jdate(df)
            df.to_csv(self.output().path,index=False, encoding='utf-8-sig')
if __name__=='__main__':
    luigi.build([cleaning()],read_path= ,dirr_path=path_2,dirr_path_2= ,local_scheduler=True)

In [ ]:
class GatherWords(luigi.Task):
    def output(self):
        return luigi.LocalTarget('text.txt')
    def run(self):
        words=['apple','orange','banana']
        return words

In [ ]:
class letterCount (luigi.Task):
    def output(self):
        return luigi.LocalTarget('letter_count.txt')
    def run(self):
        words= yield GatherWords()
        with self.output().open('w') as outfile:
            for word in words:
                outfile.write('{word}| {letter_count}\n',format(word=word,letter_count=len(word)))

In [ ]:
if __name__ == '__main__':
    luigi.build([letterCount()],local_scheduler=True)